# Single Individual Fitness Evaluation

Demonstrates how to evaluate trading fitness for one neural network individual on one epoch.

In [ ]:
import polars as pl
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
sys.path.append('../../')
import numba_ga

# Import optimized fitness functions
sys.path.append('../../src/fitnesses')
from fitness import evaluate_individual_fitness_relative

## 1. Setup & Data Loading

In [ ]:
# Load processed data with epochs
df = pl.read_parquet('../../data/mock_processed.parquet')

print(f"📊 Dataset Overview:")
print(f"   Shape: {df.shape}")
print(f"   Columns: {df.columns}")
print(f"   Epochs: {df['epoch_id'].min()} - {df['epoch_id'].max()}")
print(f"   First few rows:")
print(df.head())

In [ ]:
# Extract epoch 0 data only for this demo
epoch_id = 1
epoch = df.filter(pl.col('epoch_id') == epoch_id).sort('timestamp')

print(f"🕐 Epoch 0 Data:")
print(f"   Number of ticks: {len(epoch)}")
print(f"   Time range: {epoch['timestamp'].min():.2f} - {epoch['timestamp'].max():.2f} seconds")
print(f"   Price range: ${epoch['price'].min():.2f} - ${epoch['price'].max():.2f}")
print(f"   Duration: {(epoch['timestamp'].max() - epoch['timestamp'].min()) / 60:.1f} minutes")

In [ ]:
# Visualize the price series for epoch 0
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=epoch['timestamp'].to_numpy() / 60,  # Convert to minutes
        y=epoch['price'].to_numpy(),
        mode='lines',
        name='Price',
        line=dict(color='blue', width=1.5)
    )
)

fig.update_layout(
    title=f"Epoch {epoch_id}: Price Time Series (10 minutes)",
    xaxis_title="Time (minutes)",
    yaxis_title="Price ($)",
    width=800, height=400,
    hovermode='x unified'
)

fig.show()

## 2. Network Architecture

In [ ]:
# Define network architecture: 1 input -> 10 hidden -> 20 hidden -> 3 outputs
layer_sizes = np.array([1, 100, 200, 3], dtype=np.int64)
activations = np.array([1, 1, 2], dtype=np.int64)  # ReLU, ReLU, Sigmoid

print(f"🧠 Network Architecture:")
print(f"   Layer sizes: {layer_sizes}")
print(f"   Activations: {[numba_ga.get_activation_name(a) for a in activations]}")
print(f"   Total parameters: {numba_ga.get_total_parameters(layer_sizes)}")

# Create ONE individual
individual_params = numba_ga.initialize_individual(layer_sizes, seed=42)

print(f"\n📈 Individual Parameters:")
print(f"   Parameter count: {len(individual_params)}")
print(f"   Min: {individual_params.min():.3f}")
print(f"   Max: {individual_params.max():.3f}")
print(f"   Mean: {individual_params.mean():.3f}")
print(f"   Std: {individual_params.std():.3f}")

## 3. Fitness Function Definition

In [ ]:
from numba import njit

@njit
def evaluate_single_epoch_fitness_analysis(parameters, layer_sizes, activations,
                                          timestamps, prices):
    """
    Analysis version: Returns tracking data for visualization.
    Uses optimized fitness core but adds analysis arrays.
    """
    # Pre-allocate analysis arrays
    actions = np.zeros(len(timestamps), dtype=np.int64)
    network_outputs = np.zeros((len(timestamps), 3), dtype=np.float64)
    portfolio_history = np.zeros(len(timestamps), dtype=np.float64)
    
    # Pre-compute shared data once
    param_indices, neuron_indices = numba_ga.compute_layer_indices(layer_sizes)
    prices_normalized = prices / prices[0]  # Relative normalization
    
    # Network memory state
    input_buffer = np.zeros(1, dtype=np.float64)
    current_states = np.zeros(np.sum(layer_sizes[1:]), dtype=np.float64)
    current_time = 0.0
    
    # Trading state (all in normalized space)
    position = 0  # 0=cash, 1=stock
    buy_price_norm = 0.0
    portfolio_value = 1.0
    
    # Main evaluation loop
    for i in range(len(timestamps)):
        # Reuse input buffer (zero allocation)
        input_buffer[0] = prices_normalized[i]
        inputs = (timestamps[i], input_buffer)
        
        # Network prediction with temporal memory
        output, new_states, new_time = numba_ga.predict_individual(
            parameters, layer_sizes, activations, inputs,
            current_states, current_time, param_indices, neuron_indices
        )
        
        # Store network outputs for analysis
        network_outputs[i] = output
        
        # Optimized argmax for 3 elements (faster than np.argmax)
        action = (0 if output[0] >= output[1] and output[0] >= output[2] 
                 else 1 if output[1] >= output[2] else 2)
        
        # Trading logic in normalized space (ratios preserved)
        if position == 0 and action == 2:  # Cash → Buy
            position = 1
            buy_price_norm = prices_normalized[i]
            actual_action = 2  # BUY
        elif position == 1 and action == 0:  # Stock → Sell
            sell_price_norm = prices_normalized[i]
            portfolio_value *= (sell_price_norm / buy_price_norm)
            position = 0
            actual_action = 0  # SELL
        else:
            actual_action = 1  # HOLD (forced or chosen)
        
        actions[i] = actual_action
        portfolio_history[i] = portfolio_value
        
        # Update network state
        current_states = new_states
        current_time = new_time
    
    # Close position if holding stock at epoch end
    if position == 1:
        final_price_norm = prices_normalized[-1]
        portfolio_value *= (final_price_norm / buy_price_norm)
        portfolio_history[-1] = portfolio_value
    
    return portfolio_value, actions, network_outputs, portfolio_history

print("✅ Optimized fitness function with analysis tracking")

## 4. Run Evaluation

In [ ]:
# Extract arrays for numba function
timestamps = epoch['timestamp'].to_numpy()
prices = epoch['price'].to_numpy()

print(f"🚀 Running fitness evaluation...")
print(f"   Processing {len(timestamps)} ticks")
print(f"   Time span: {timestamps[-1] - timestamps[0]:.1f} seconds")

# Run the evaluation with BOTH methods for comparison

# 1. Optimized fitness (production version)
param_indices, neuron_indices = numba_ga.compute_layer_indices(layer_sizes)
prices_normalized = prices / prices[0]  # Relative normalization

optimized_fitness = evaluate_individual_fitness_relative(
    individual_params, layer_sizes, activations,
    timestamps, prices_normalized,
    param_indices, neuron_indices
)

# 2. Analysis version (for visualization)
final_fitness, actions, network_outputs, portfolio_history = evaluate_single_epoch_fitness_analysis(
    individual_params, layer_sizes, activations, timestamps, prices
)

print(f"\n📊 Evaluation Results:")
print(f"   Optimized fitness: {optimized_fitness:.4f}")
print(f"   Analysis fitness:  {final_fitness:.4f}")
print(f"   Match: {'✅' if abs(optimized_fitness - final_fitness) < 1e-10 else '❌'}")
print(f"   Total return: {(final_fitness - 1.0) * 100:.2f}%")

# Count actions
action_counts = np.bincount(actions, minlength=3)
print(f"\n🎯 Action Summary:")
print(f"   Sells (0): {action_counts[0]} ({action_counts[0]/len(actions)*100:.1f}%)")
print(f"   Holds (1): {action_counts[1]} ({action_counts[1]/len(actions)*100:.1f}%)")
print(f"   Buys (2): {action_counts[2]} ({action_counts[2]/len(actions)*100:.1f}%)")

In [ ]:
# Demonstrate the performance improvement and scalability
import time

print("⚡ PERFORMANCE COMPARISON")
print("=" * 40)
for _ in range(10):
      optimized_fitness = evaluate_individual_fitness_relative(
        individual_params, layer_sizes, activations,
        timestamps, prices_normalized,
        param_indices, neuron_indices
    )


# Time the optimized version (multiple runs for accuracy)
start_time = time.time()
for _ in range(1000):
    optimized_fitness = evaluate_individual_fitness_relative(
        individual_params, layer_sizes, activations,
        timestamps, prices_normalized,
        param_indices, neuron_indices
    )
optimized_time = (time.time() - start_time) / 100

# Time the analysis version
start_time = time.time()
for _ in range(100):  # Fewer runs as it's slower
    final_fitness, actions, network_outputs, portfolio_history = evaluate_single_epoch_fitness_analysis(
        individual_params, layer_sizes, activations, timestamps, prices
    )
analysis_time = (time.time() - start_time) / 10

print(f"\n🏃 Speed Results:")
print(f"   Optimized function: {optimized_time*1000:.2f} ms per evaluation")
print(f"   Analysis function:  {analysis_time*1000:.2f} ms per evaluation")
print(f"   Speedup: {analysis_time/optimized_time:.1f}x faster")

print(f"\n🧠 Key Optimizations Applied:")
print(f"   ✅ Relative normalization (no hardcoded 1000)")
print(f"   ✅ Zero runtime allocation (reused arrays)")
print(f"   ✅ Manual argmax (faster for 3 elements)")
print(f"   ✅ Pre-computed indices (shared)")
print(f"   ✅ Scale-invariant design")

print(f"\n🚀 Production Ready:")
print(f"   • Can evaluate ~{int(1.0/(optimized_time+1e-10)):,} individuals/second")
print(f"   • Ready for population-level parallelization")
print(f"   • Memory efficient for large-scale GA")

## 5. Results Visualization

In [ ]:
# Create comprehensive 4-panel visualization
fig = make_subplots(
    rows=4, cols=1,
    subplot_titles=[
        'Price Over Time with Trading Actions',
        'Network Raw Outputs (3 neurons)',
        'Actual Actions Taken (with constraints)',
        'Portfolio Value Evolution'
    ],
    vertical_spacing=0.08,
    specs=[[{"secondary_y": False}]] * 4
)

# Convert timestamps to minutes for better readability
time_minutes = timestamps / 60

# 1. Price + Trading Events
fig.add_trace(
    go.Scatter(
        x=time_minutes,
        y=prices,
        mode='lines',
        name='Price',
        line=dict(color='blue', width=1.5)
    ),
    row=1, col=1
)

# Add buy/sell markers
buy_mask = actions == 2
sell_mask = actions == 0

if np.any(buy_mask):
    fig.add_trace(
        go.Scatter(
            x=time_minutes[buy_mask],
            y=prices[buy_mask],
            mode='markers',
            name='BUY',
            marker=dict(color='green', size=8, symbol='triangle-up')
        ),
        row=1, col=1
    )

if np.any(sell_mask):
    fig.add_trace(
        go.Scatter(
            x=time_minutes[sell_mask],
            y=prices[sell_mask],
            mode='markers',
            name='SELL',
            marker=dict(color='red', size=8, symbol='triangle-down')
        ),
        row=1, col=1
    )

# 2. Network Raw Outputs
colors = ['red', 'orange', 'green']
names = ['Sell Signal', 'Hold Signal', 'Buy Signal']
for i in range(3):
    fig.add_trace(
        go.Scatter(
            x=time_minutes,
            y=network_outputs[:, i],
            mode='lines',
            name=names[i],
            line=dict(color=colors[i], width=1),
            opacity=0.8
        ),
        row=2, col=1
    )

# 3. Actual Actions
action_names = ['SELL', 'HOLD', 'BUY']
action_colors = ['red', 'gray', 'green']

fig.add_trace(
    go.Scatter(
        x=time_minutes,
        y=actions,
        mode='markers',
        name='Actions',
        marker=dict(
            color=[action_colors[a] for a in actions],
            size=3
        )
    ),
    row=3, col=1
)

# 4. Portfolio Value
fig.add_trace(
    go.Scatter(
        x=time_minutes,
        y=portfolio_history,
        mode='lines',
        name='Portfolio Value',
        line=dict(color='purple', width=2)
    ),
    row=4, col=1
)

# Add horizontal line at portfolio = 1.0
fig.add_hline(y=1.0, line_dash="dash", line_color="gray", opacity=0.5, row=4, col=1)

# Update layout
fig.update_xaxes(title_text="Time (minutes)", row=4, col=1)
fig.update_yaxes(title_text="Price ($)", row=1, col=1)
fig.update_yaxes(title_text="Output Value", row=2, col=1)
fig.update_yaxes(title_text="Action", row=3, col=1, tickvals=[0, 1, 2], ticktext=['SELL', 'HOLD', 'BUY'])
fig.update_yaxes(title_text="Portfolio Value", row=4, col=1)

fig.update_layout(
    title="Single Individual Trading Performance Analysis",
    height=1000,
    width=900,
    showlegend=True
)

fig.show()

## 6. Analysis

In [ ]:
# Detailed analysis of trading performance
print("📈 COMPREHENSIVE TRADING ANALYSIS")
print("=" * 50)

# Basic metrics
print(f"\n💰 Financial Performance:")
print(f"   Starting value: $1.00")
print(f"   Final value: ${final_fitness:.4f}")
print(f"   Total return: {(final_fitness - 1.0) * 100:.2f}%")

# Trading frequency
num_trades = action_counts[0] + action_counts[2]  # buys + sells
print(f"\n📊 Trading Activity:")
print(f"   Total trades: {num_trades}")
print(f"   Buy orders: {action_counts[2]}")
print(f"   Sell orders: {action_counts[0]}")
print(f"   Trading frequency: {num_trades / len(timestamps) * 100:.2f}% of ticks")

# Network behavior analysis
print(f"\n🧠 Network Behavior:")
print(f"   Raw output ranges:")
for i, name in enumerate(['Sell', 'Hold', 'Buy']):
    print(f"     {name}: {network_outputs[:, i].min():.3f} to {network_outputs[:, i].max():.3f}")

# Position analysis
position_changes = np.diff(np.concatenate([[0], actions]))
buy_points = np.where(actions == 2)[0]
sell_points = np.where(actions == 0)[0]

print(f"\n🔄 Position Management:")
print(f"   Buy signals executed: {len(buy_points)}")
print(f"   Sell signals executed: {len(sell_points)}")

if len(buy_points) > 0 and len(sell_points) > 0:
    print(f"   First buy at: ${prices[buy_points[0]]:.2f}")
    print(f"   Last sell at: ${prices[sell_points[-1]]:.2f}")

# Constraint effectiveness
desired_actions = np.argmax(network_outputs, axis=1)
constraint_violations = np.sum(desired_actions != actions)
print(f"\n⚖️ Trading Constraints:")
print(f"   Times constrained: {constraint_violations} ({constraint_violations/len(actions)*100:.1f}%)")
print(f"   Times unconstrained: {len(actions) - constraint_violations} ({(len(actions) - constraint_violations)/len(actions)*100:.1f}%)")

# Risk assessment
portfolio_volatility = np.std(portfolio_history)
max_drawdown = np.max(portfolio_history) - np.min(portfolio_history)

print(f"\n⚠️ Risk Metrics:")
print(f"   Portfolio volatility: {portfolio_volatility:.4f}")
print(f"   Max drawdown: {max_drawdown:.4f}")
print(f"   Final position: {'HOLDING STOCK' if (action_counts[2] > action_counts[0]) else 'CASH'}")

print(f"\n✅ EVALUATION COMPLETE")
print("=" * 50)

In [ ]:
import cProfile

profiler = cProfile.Profile()
profiler.enable()

for _ in range(100):
    optimized_fitness = evaluate_individual_fitness_relative(
        individual_params, layer_sizes, activations,
        timestamps, prices_normalized,
        param_indices, neuron_indices
    )

profiler.disable()
profiler.print_stats(sort='cumulative')
